#The following sites have been referred to for the implementation of this code.

https://github.com/pat-coady/tiny_imagenet

http://solarisailab.com/archives/2325

# Rank-1 CNN on CIFAR10 dataset

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
import re
import os

# use matplotlib as inline
%matplotlib inline 
from functools import partial
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.keras._impl.keras.datasets.cifar10 import load_data

# for reading the next batch
def next_batch(num, data, labels): 
    ''' 
    Return a total of `num` random samples and labels.  
    ''' 
    idx = np.arange(0 , len(data)) 
    np.random.shuffle(idx) 
    idx = idx[:num] 
    data_shuffle = [data[i] for i in idx] 
    labels_shuffle = [labels[i] for i in idx] 

 
    return np.asarray(data_shuffle), np.asarray(labels_shuffle) 

In [3]:
 #configuration for GPU to use 90% of its capacity
config = tf.ConfigProto()
config.gpu_options.allocator_type ='BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.90

In [4]:
x = tf.placeholder("float", [None, 32, 32, 3])

phase = tf.placeholder(tf.bool, name='phase') #whether batch normalization is in its training or test mode

# conv1_1
input_num = 3  #number of input channels
dim = 3 # Size of the 1-D vectors
num = 8 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num   # number of filters = number of outputs

W_list1_1 = [] # list for W filter

P1_1 = tf.get_variable("P1_1", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
Q1_1 = tf.get_variable("Q1_1", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
T1_1 = tf.get_variable("T1_1", shape=[output_num,input_num,1,1],
           initializer=tf.contrib.layers.xavier_initializer())

k=0
for i in range(num): 
    for j in range(num):
      P1_1_temp = tf.reshape(P1_1[i], [dim, 1])
      Q1_1_temp = tf.reshape(Q1_1[j], [1, dim])
      Mul_temp1_1_a = tf.matmul(P1_1_temp, Q1_1_temp)
      Mul_temp1_1_b = tf.reshape(Mul_temp1_1_a, [1, Mul_temp1_1_a.shape[0], Mul_temp1_1_a.shape[1]])
      Mul_temp1_1_c = T1_1[k]*Mul_temp1_1_b
      k = k + 1
      W_list1_1.append(Mul_temp1_1_c)
   
    
W1_1 = tf.transpose(tf.stack(W_list1_1, axis=0))

L1 = tf.nn.conv2d(x, W1_1, strides=[1, 1, 1, 1], padding='SAME')

Bias1_1 = tf.Variable(tf.constant(0.0, shape=[output_num], dtype=tf.float32),
                                 trainable=True, name='Bias1_1')
L1 = tf.nn.bias_add(L1, Bias1_1)
L1 = tf.nn.relu(L1)
# batch normalization
L1 = tf.layers.batch_normalization(L1, training=phase)

In [5]:
print(x.shape)

(?, 32, 32, 3)


In [6]:
# conv1_2
input_num = 64  #number of input channels
dim = 3 # Size of the 1-D vectors
num = 8 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num # number of filters = number of outputs

W_list1_2 = [] # list for W filter
#W = tf.Variable(tf.random_uniform([10,3,3], -0.5, 0.5))

P1_2 = tf.get_variable("P1_2", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
Q1_2 = tf.get_variable("Q1_2", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
T1_2 = tf.get_variable("T1_2", shape=[output_num,input_num,1,1],
           initializer=tf.contrib.layers.xavier_initializer())

k=0
for i in range(num): 
    for j in range(num):
      P1_2_temp = tf.reshape(P1_2[i], [dim, 1])
      Q1_2_temp = tf.reshape(Q1_2[j], [1, dim])
      Mul_temp1_2_a = tf.matmul(P1_2_temp, Q1_2_temp)
      Mul_temp1_2_b = tf.reshape(Mul_temp1_2_a, [1, Mul_temp1_2_a.shape[0], Mul_temp1_2_a.shape[1]])
      Mul_temp1_2_c = T1_2[k]*Mul_temp1_2_b
      k = k + 1
      W_list1_2.append(Mul_temp1_2_c)

W1_2 = tf.transpose(tf.stack(W_list1_2, axis=0))

L1 = tf.nn.conv2d(L1, W1_2, strides=[1, 1, 1, 1], padding='SAME')
Bias1_2 = tf.Variable(tf.constant(0.0, shape=[output_num], dtype=tf.float32),
                                 trainable=True, name='Bias1_2')
L1 = tf.nn.bias_add(L1, Bias1_2)
# batch normalization
#L1 = tf.layers.batch_normalization(L1, training=phase)

L1 = tf.nn.relu(L1)
# Max_Pool1
# 입력 : (N, 160, 160, 64)  --> 출력 : (N, 80, 80, 64)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

L1 = tf.nn.dropout(L1, 0.5) 

In [8]:
# conv2_1

input_num = 64    #number of input channels
dim = 3 # Size of the 1-D vectors
num = 12 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs

W_list2_1 = [] # list for W filter

P2_1 = tf.get_variable("P2_1", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
Q2_1 = tf.get_variable("Q2_1", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
T2_1 = tf.get_variable("T2_1", shape=[output_num,input_num,1,1],
           initializer=tf.contrib.layers.xavier_initializer())

k=0
for i in range(num): 
    for j in range(num):
      P2_1_temp = tf.reshape(P2_1[i], [dim, 1])
      Q2_1_temp = tf.reshape(Q2_1[j], [1, dim])
      Mul_temp2_1_a = tf.matmul(P2_1_temp, Q2_1_temp)
      Mul_temp2_1_b = tf.reshape(Mul_temp2_1_a, [1, Mul_temp2_1_a.shape[0], Mul_temp2_1_a.shape[1]])
      Mul_temp2_1_c = T2_1[k]*Mul_temp2_1_b
      k = k + 1
      W_list2_1.append(Mul_temp2_1_c)

W2_1 = tf.transpose(tf.stack(W_list2_1, axis=0))

L2 = tf.nn.conv2d(L1, W2_1, strides=[1, 1, 1, 1], padding='SAME')

Bias2_1 = tf.Variable(tf.constant(0.0, shape=[output_num], dtype=tf.float32),
                                 trainable=True, name='Bias2_1')
L2 = tf.nn.bias_add(L2, Bias2_1)

L2 = tf.nn.relu(L2)

# batch normalization
L2 = tf.layers.batch_normalization(L2, training=phase)

In [10]:
# conv2_2

input_num = 144    #number of input channels
dim = 3 # Size of the 1-D vectors
num = 12 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs

W_list2_2 = [] # list for W filter

P2_2 = tf.get_variable("P2_2", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
Q2_2 = tf.get_variable("Q2_2", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
T2_2 = tf.get_variable("T2_2", shape=[output_num,input_num,1,1],
           initializer=tf.contrib.layers.xavier_initializer())

k=0
for i in range(num): 
    for j in range(num):
      P2_2_temp = tf.reshape(P2_2[i], [dim, 1])
      Q2_2_temp = tf.reshape(Q2_2[j], [1, dim])
      Mul_temp2_2_a = tf.matmul(P2_2_temp, Q2_2_temp)
      Mul_temp2_2_b = tf.reshape(Mul_temp2_2_a, [1, Mul_temp2_2_a.shape[0], Mul_temp2_2_a.shape[1]])
      Mul_temp2_2_c = T2_2[k]*Mul_temp2_2_b
      k = k + 1
      W_list2_2.append(Mul_temp2_2_c)

W2_2 = tf.transpose(tf.stack(W_list2_2, axis=0))

L2 = tf.nn.conv2d(L2, W2_2, strides=[1, 1, 1, 1], padding='SAME')
Bias2_2 = tf.Variable(tf.constant(0.0, shape=[output_num], dtype=tf.float32),
                                 trainable=True, name='Bias2_2')
L2 = tf.nn.bias_add(L2, Bias2_2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, 0.5) 

In [12]:
# conv3_1

input_num = 144    #number of input channels
dim = 3 # Size of the 1-D vectors
num = 16 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num # number of filters = number of outputs

W_list3_1 = [] # list for W filter

P3_1 = tf.get_variable("P3_1", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
Q3_1 = tf.get_variable("Q3_1", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
T3_1 = tf.get_variable("T3_1", shape=[output_num,input_num,1,1],
           initializer=tf.contrib.layers.xavier_initializer())

k=0
for i in range(num): 
    for j in range(num):
      P3_1_temp = tf.reshape(P3_1[i], [dim, 1])
      Q3_1_temp = tf.reshape(Q3_1[j], [1, dim])
      Mul_temp3_1_a = tf.matmul(P3_1_temp, Q3_1_temp)
      Mul_temp3_1_b = tf.reshape(Mul_temp3_1_a, [1, Mul_temp3_1_a.shape[0], Mul_temp3_1_a.shape[1]])
      Mul_temp3_1_c = T3_1[k]*Mul_temp3_1_b
      k = k + 1
      W_list3_1.append(Mul_temp3_1_c)

W3_1 = tf.transpose(tf.stack(W_list3_1, axis=0))

L3 = tf.nn.conv2d(L2, W3_1, strides=[1, 1, 1, 1], padding='SAME')
Bias3_1 = tf.Variable(tf.constant(0.0, shape=[output_num], dtype=tf.float32),
                                 trainable=True, name='Bias3_1')
L3 = tf.nn.bias_add(L3, Bias3_1)

L3 = tf.nn.relu(L3)
# batch normalization
L3 = tf.layers.batch_normalization(L3, training=phase)

In [14]:
# conv3_2

input_num = 256    #number of input channels
dim = 3  # Size of the 1-D vectors 
num = 16 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num # number of filters = number of outputs

W_list3_2 = [] # list for W filter

P3_2 = tf.get_variable("P3_2", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
Q3_2 = tf.get_variable("Q3_2", shape=[num,dim],
           initializer=tf.contrib.layers.xavier_initializer())
T3_2 = tf.get_variable("T3_2", shape=[output_num,input_num,1,1],
           initializer=tf.contrib.layers.xavier_initializer())

k=0
for i in range(num): 
    for j in range(num):
      P3_2_temp = tf.reshape(P3_2[i], [dim, 1])
      Q3_2_temp = tf.reshape(Q3_2[j], [1, dim])
      Mul_temp3_2_a = tf.matmul(P3_2_temp, Q3_2_temp)
      Mul_temp3_2_b = tf.reshape(Mul_temp3_2_a, [1, Mul_temp3_2_a.shape[0], Mul_temp3_2_a.shape[1]])
      Mul_temp3_2_c = T3_2[k]*Mul_temp3_2_b
      k = k + 1
      W_list3_2.append(Mul_temp3_2_c)

W3_2 = tf.transpose(tf.stack(W_list3_2, axis=0))

L3 = tf.nn.conv2d(L3, W3_2, strides=[1, 1, 1, 1], padding='SAME')

Bias3_2 = tf.Variable(tf.constant(0.0, shape=[output_num], dtype=tf.float32),
                                 trainable=True, name='Bias3_2')
L3 = tf.nn.bias_add(L3, Bias3_2)
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, 0.5) 

In [16]:
def dense(inputs, units, name=None): 
#"""3x3 conv layer: ReLU + He initialization""" 
# He initialization: normal dist with stdev = sqrt(2.0/fan-in) 
    stddev = np.sqrt(2 / int(inputs.shape[1])) 
    out = tf.layers.dense(inputs, units, 
                           kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1.0), 
                           name=name) 

    return out 

def dense_relu(inputs, units, name=None): 
    # """3x3 conv layer: ReLU + He initialization"""   
    # He initialization: normal dist with stdev = sqrt(2.0/fanin) 
    stddev = np.sqrt(2 / int(inputs.shape[1])) 
    out = tf.layers.dense(inputs, units, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(1.0), 
                         name=name) 
  
    return out 

def dense_batch_relu(x, output, phase, scope):
    with tf.variable_scope(scope):
        h1 = tf.contrib.layers.fully_connected(x, output, 
                                               activation_fn=None,
                                               scope='dense')
        h2 = tf.contrib.layers.batch_norm(h1, 
                                          center=True, scale=True, 
                                          is_training=phase,
                                          scope='bn')
        return tf.nn.relu(h2, 'relu')

In [17]:
y_ = tf.placeholder("float", [None,10])

out = tf.contrib.layers.flatten(L3)

In [18]:
out = dense_batch_relu(out, 1024, phase,'fc1')
out = tf.nn.dropout(out, 0.5) 
out = dense_batch_relu(out, 512, phase,'fc2')
out = tf.nn.dropout(out, 0.5) 

class_number = 10
logits = dense(out, class_number, 'fc3')  # class number =2

In [19]:
# Downloading the CIFAR-10 dataset and read them in.
def normalize(X_train,X_test): 
        #this function normalize inputs for zero mean and unit variance 
        # it is used when training a model. 
        # Input: training set and test set 
        # Output: normalized training set and test set according to the trianing set statistics. 
        X_train = X_train-128
        X_test =  X_test-128
#        mean = np.mean(X_train,axis=(0,1,2,3)) 
#        std = np.std(X_train, axis=(0, 1, 2, 3)) 
#        X_train = (X_train-mean)/(std+1e-7) 
#        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test 

(x_train, y_train), (x_test, y_test) = load_data() 
x_train = x_train.astype('float32') 
x_test = x_test.astype('float32') 
x_train, x_test = normalize(x_train, x_test) 

# Labels into One-hot Encoding form
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1) 
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

In [20]:
#########
# Result
########
is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [21]:
# Loss function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))

train_batch_size = 100
test_batch_size = 100
total_train_batch = int(50000/train_batch_size)
total_test_batch = int(10000/test_batch_size)

saver = tf.train.Saver(tf.global_variables())
global_step = tf.Variable(0, trainable=False, name='global_step')

number_of_epochs = 10000
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
     # Ensures that we execute the update_ops before performing the train_step
     train_step = tf.train.AdamOptimizer(0.001).minimize(cost, global_step=global_step)
    

sess = tf.InteractiveSession() 
sess.run(tf.global_variables_initializer())

for epoch in range(number_of_epochs):
    total_cost = 0
    total_accuracy = 0
#    if ckpt:
#        saver.restore(sess, ckpt.model_checkpoint_path)
#    else:
#        sess.run(tf.global_variables_initializer())
    start_time = time.time()
    for i in range(total_train_batch):
        
        batch = next_batch(train_batch_size, x_train, y_train_one_hot.eval())
        _, cost_val = sess.run([train_step,cost], feed_dict={x: batch[0], y_: batch[1], phase: True})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),'Avg. cost =', '{:.3f}'.format(total_cost))
    print("--- %s seconds ---" % (time.time() - start_time))
    
    for i in range(total_test_batch):

        test_batch = next_batch(test_batch_size, x_test, y_test_one_hot.eval()) 
        acc = sess.run(accuracy, feed_dict={x: test_batch[0], y_: test_batch[1], phase: False})
        total_accuracy += acc
    
    mean_accuracy = total_accuracy/total_test_batch
    print('Accuracy:', mean_accuracy) # Mean accuracy for test batches
    
# save weights as checkpoint file.    
#    saver.save(sess, './cnn(cifar10)/dnn.ckpt', global_step=global_step)


print('End of optimization!')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch: 0001 Avg. cost = 856.784
--- 94.62800216674805 seconds ---
Accuracy: 0.2781000010669231
Epoch: 0002 Avg. cost = 555.322
--- 80.51699876785278 seconds ---
Accuracy: 0.49449999779462817
Epoch: 0003 Avg. cost = 474.815
--- 80.92648458480835 seconds ---
Accuracy: 0.6300000029802323
Epoch: 0004 Avg. cost = 427.712
--- 80.70384764671326 seconds ---
Accuracy: 0.6370999997854233
Epoch: 0005 Avg. cost = 399.844
--- 80.90805196762085 seconds ---
Accuracy: 0.6721999990940094
Epoch: 0006 Avg. cost = 377.702
--- 80.89074444770813 seconds ---
Accuracy: 0.7115000027418137
Epoch: 0007 Avg. cost = 361.640
--- 80.9800271987915 seconds ---
Accuracy: 0.7068000024557114
Epoch: 0008 Avg. cost = 337.694
--- 80.63636898994446 seconds ---
Accuracy: 0.7175
Epoch: 0009 Avg. cost = 328.702
--- 80.5608639717102 seconds

KeyboardInterrupt: 